In [41]:
import torch
import numpy as np

def trans_t(t):

    return torch.Tensor([
    [1.,0.,0.,0.],
    [0.,1.,0.,0.],
    [0.,0.,1.,t],
    [0.,0.,0.,1.],
])

def rot_phi(phi):
    
     return torch.Tensor([
    [1.,0.,0.,0.],
    [0.,torch.cos(phi),-torch.sin(phi),0.],
    [0.,torch.sin(phi), torch.cos(phi),0.],
    [0.,0.,0.,1.],])


def rot_theta(th):

    return torch.Tensor([
        [torch.cos(th),0.,-torch.sin(th),0.],
        [0.,1.,0.,0.],
        [torch.sin(th),0, torch.cos(th),0],
        [0.,0.,0.,1.],
    ])    


def pose_spherical(theta, phi, radius):
    c2w = trans_t(torch.Tensor([radius]))
    c2w = rot_phi(torch.Tensor([phi/180.*np.pi])) @ c2w
    c2w = rot_theta(torch.Tensor([theta/180.*np.pi])) @ c2w
    c2w = torch.Tensor([[-1.,0.,0.,0.],[0.,0.,1.,0.],[0.,1.,0.,0.],[0.,0.,0.,1.]]) @ c2w
    return c2w.numpy()



In [59]:
def get_rays_rendering(c2w):

    H = 400
    W = 400
    f = 1333.33

    rays_o = np.zeros((H*W, 3))
    rays_d = np.zeros((H*W, 3))
    u = np.arange(W)
    v = np.arange(H)
    u, v = np.meshgrid(u, v)
    dirs = np.stack((u - W / 2, -(v - H / 2), - np.ones_like(u) * f), axis=-1)
    dirs = (c2w[:3, :3] @ dirs[..., None]).squeeze(-1)
    dirs = dirs / np.linalg.norm(dirs, axis=-1, keepdims=True)
        
    rays_d= dirs.reshape(-1, 3)
    rays_o+= c2w[:3, 3]

    return rays_o, rays_d

In [60]:
device = 'cpu'
tn = 8
tf = 12

model = torch.load("model_voxel")


from rendering import rendering

frames = []
for th in np.linspace(0., 360., 120, endpoint=False):
    c2w = pose_spherical(0., -30., 4.)
    rays_o, rays_d = get_rays_rendering(c2w[:3,:4])
    print(rays_o.shape,rays_d)
    img = rendering(model, torch.from_numpy(rays_o).to(device), torch.from_numpy(rays_d).to(device), tn, tf, nb_bins=100, device=device)
    frames.append(img.reshape(400, 400, 3).data.cpu().numpy().astype(np.uint8))



(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160000, 3)
(160000, 3) (160

KeyboardInterrupt: 

In [48]:
import imageio
f = 'video.mp4'
imageio.mimwrite(f, frames, fps=30, quality=7)